In [0]:
import pyspark
from pyspark.sql import DataFrameWriter
import pandas as pd
from pyspark.sql.functions import *

In [0]:
#dbutils.secrets.listScopes()

In [0]:
#dbutils.secrets.list(scope='healthcarescope')

In [0]:
spark.conf.set("fs.azure.account.key.carehealthcareblobsa.dfs.core.windows.net",
    dbutils.secrets.get(scope="healthcarescope", key="blobaccesskey"))

In [0]:
#display(dbutils.fs.ls("abfss://rawhealthcaredata@carehealthcareblobsa.dfs.core.windows.net"))

In [0]:
subgroup_data = spark.read.csv("abfss://rawhealthcaredata@carehealthcareblobsa.dfs.core.windows.net/subgroup.csv", header=True, inferSchema=True)

In [0]:
#subgroup_data.show(5,False)

In [0]:
#subgroup_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in subgroup_data.columns]).show()

In [0]:
#check to duplicates
#drop the duplicates

In [0]:
#subgroup_data.groupby(['subgrp_sk','subgrp_name','monthly_premium','subgrp_id']).count().where("count > 1").show() #check the duplicates

In [0]:
subgroup_data = subgroup_data.dropDuplicates()

In [0]:
#subgroup_data.show(6,False)

In [0]:
subgroup_data = subgroup_data.withColumn("subgrp_id",split(subgroup_data['subgrp_id'],","))

In [0]:
#subgroup_data.show(6,False)

In [0]:
#explode ---> transfrom each element of the list to row

In [0]:
subgroup_data = subgroup_data.withColumn("subgrp_id", explode(subgroup_data['subgrp_id']))

In [0]:
subgroup_data.show(50,False)

+---------+-------------------+---------------+---------+
|subgrp_sk|subgrp_name        |monthly_premium|subgrp_id|
+---------+-------------------+---------------+---------+
|S108     |Infectious disease |1500           |GRP130   |
|S108     |Infectious disease |1500           |GRP104   |
|S108     |Infectious disease |1500           |GRP109   |
|S104     |Therapy            |1500           |GRP103   |
|S104     |Therapy            |1500           |GRP113   |
|S104     |Therapy            |1500           |GRP123   |
|S104     |Therapy            |1500           |GRP133   |
|S104     |Therapy            |1500           |GRP143   |
|S105     |Allergies          |2300           |GRP153   |
|S105     |Allergies          |2300           |GRP104   |
|S105     |Allergies          |2300           |GRP114   |
|S105     |Allergies          |2300           |GRP124   |
|S109     |Hereditary         |2000           |GRP102   |
|S109     |Hereditary         |2000           |GRP112   |
|S109     |Her

In [0]:
#loading df into stagging
#stagging_path = "abfss://stagginghealthcaredata@carehealthcareblobsa.dfs.core.windows.net"
#subgroup_data.write.format("csv").option("header", "true").mode("overwrite").option("path",stagging_path).save()

In [0]:

output_container_path = "abfss://stagginghealthcaredata@carehealthcareblobsa.dfs.core.windows.net"
output_blob_folder = "stagingdatasets/" 
subgroup_data.coalesce(1).write.mode("overwrite").option("header", "true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path, "%s/subgroupdatastagging.csv" % output_container_path)

True

In [0]:
"""
output_container_path = "abfss://stagingdatasets@careblobstore.dfs.core.windows.net"
output_blob_folder = "stagingdatasets/wrangled_data_folder" 
(data
 .coalesce(1)
 .write
 .mode("overwrite")
 .option("header", "true")
 .format("com.databricks.spark.csv")
 .save(output_blob_folder))

files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path, "%s/staging_subgroup.csv" % output_container_path)
"""